In [1]:
import datajoint as dj
import numpy as np

ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

Connecting celiib@10.28.0.34:3306


In [2]:
schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')

In [ ]:
ta3p100.NeuriteSkeletons()

In [ ]:
#NeuriteSkeletons
@schema
class NeuriteRawSkeleton(dj.Computed):
    definition="""
    -> ta3p100.Mesh
    time_updated      :timestamp    # the time at which the skeleton was generated
    ---
    n_edges   :int unsigned #number of edges stored
    edges     :longblob #array storing edges on each row
    n_bodies    :tinyint unsigned #the amount of segments the neurite was originally split into
    lagest_mesh_perc : float #the percentage of the entire mesh that the largest submesh makes up
     
    """

In [ ]:
ta3p100.NeuriteRawSkeleton()#.drop()

In [3]:
ta3p100.NeuronRawSkeleton()

segmentation segmentation id,segment_id segment id unique within each Segmentation,time_updated the time at which the component labels were updated,n_edges number of edges stored,edges array storing edges on each row,n_vertices number of vertices in mesh filtered of error segments,n_triangles number of faces in mesh filtered of error segments,vertices mesh data for vertices in mesh filtered of error segments,triangles mesh data for faces in mesh filtered of error segments
2,648518346341352006,2019-03-29 18:55:10,1420,=BLOB=,25455,50819,=BLOB=,=BLOB=
2,648518346341352223,2019-03-29 18:55:22,1080,=BLOB=,39026,77823,=BLOB=,=BLOB=
2,648518346341353019,2019-03-29 18:55:25,722,=BLOB=,8805,17592,=BLOB=,=BLOB=
2,648518346341353186,2019-03-29 18:55:40,1220,=BLOB=,43636,86901,=BLOB=,=BLOB=
2,648518346341353521,2019-03-29 18:55:41,0,=BLOB=,0,0,=BLOB=,=BLOB=
2,648518346341353574,2019-03-29 18:55:58,1598,=BLOB=,56775,113343,=BLOB=,=BLOB=
2,648518346341353607,2019-03-29 18:56:19,1152,=BLOB=,68234,136186,=BLOB=,=BLOB=


In [17]:
(len(ta3p100.NeuriteRawSkeleton()),len(ta3p100.Mesh() & ta3p100.NeuriteRevised() & ta3p100.CurrentSegmentation))

(32232, 32238)

In [20]:
#the ones not finished after running script for neurites
(ta3p100.Mesh() & ta3p100.NeuriteRevised() & ta3p100.CurrentSegmentation) - (ta3p100.NeuriteRawSkeleton())

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices)
2,648518346341375543,21323,42430,=BLOB=,=BLOB=
2,648518346341379287,10126,20136,=BLOB=,=BLOB=
2,648518346341414637,28637,57050,=BLOB=,=BLOB=
2,648518346342796533,55575,110842,=BLOB=,=BLOB=
2,648518346342798131,81788,163158,=BLOB=,=BLOB=
2,648518346349380626,11994,23908,=BLOB=,=BLOB=


In [21]:
#the ones not finished after running script for neurons
ns_table = ((dj.U("segmentation","segment_id") & ta3p100.CoarseLabelFinal.proj()) 
     + (dj.U("segmentation","segment_id") & ta3p100.CoarseLabelOrphan.proj()))
ns_table - ta3p100.NeuronRawSkeleton()

segmentation segmentation id,segment_id segment id unique within each Segmentation
2,648518346349501787
2,648518346342803193


In [22]:
#the ones not finished after running script for neurons
ns_table = ((dj.U("segmentation","segment_id") & ta3p100.CoarseLabelFinal.proj()) 
     + (dj.U("segmentation","segment_id") & ta3p100.CoarseLabelOrphan.proj()))
ns_table - ta3p100.NeuronRawSkeletonCorrespondence()

segmentation segmentation id,segment_id segment id unique within each Segmentation
2,648518346349501787
2,648518346342803193


In [24]:
ta3p100.NeuriteRawSkeleton() & "segment_id=648518346349507130"

segmentation segmentation id,segment_id segment id unique within each Segmentation,time_updated the time at which the skeleton was generated,n_edges number of edges stored,edges array storing edges on each row,n_bodies the amount of segments the neurite was originally split into,lagest_mesh_perc the percentage of the entire mesh that the largest submesh makes up
2,648518346349507130,2019-03-29 23:45:06,316,=BLOB=,21,0.404139


In [9]:
@schema
class NeuronRawSkeleton(dj.Computed):
    definition="""
    -> ta3p100.Mesh
    time_updated      :timestamp    # the time at which the component labels were updated
    ---
    n_edges   :int unsigned #number of edges stored
    edges     :longblob #array storing edges on each row
    n_vertices   :int unsigned #number of vertices in mesh filtered of error segments
    n_triangles  :int unsigned #number of faces in mesh filtered of error segments
    vertices     :longblob #mesh data for vertices in mesh filtered of error segments
    triangles    :longblob #mesh data for faces in mesh filtered of error segments
     
    """

In [ ]:
@schema
class NeuronRawSkeletonCorrespondence(dj.Manual):
    definition="""
    -> ta3p100.Mesh
    time_updated      :timestamp    # the time at which the component labels were updated
    ---
    n_correspondence   :int unsigned #number of mappings from skeleton vert to original surface vert 
    correspondence     :longblob #array storing mapping of every skeleton vert to original surface vert  
    """

In [ ]:
#ta3p100.NeuronRawSkeletonCorrespondence.drop()

In [ ]:
#resetting the tables
#ta3p100.NeuronRawSkeletonCorrespondence.delete()
#ta3p100.NeuronRawSkeleton.delete()

In [ ]:
(schema.jobs & "table_name='__neurite_raw_skeleton'")#.delete()

In [ ]:
""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)


import csv
import math
import numpy as np

def read_raw_skeleton(file_path):
    edges = list()
    with open(file_path) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        for i,row in enumerate(reader):
            v1 = (float(row[1]),float(row[2]),float(row[3]))
            v2 = (float(row[4]),float(row[5]),float(row[6]))
            edges.append((v1,v2))
    return np.array(edges).astype(float)

In [ ]:


#TESTING OUT HOW TO IMPORT THE NEW ARRAY:
edges_file = "elephant_new_2_edges-2by2.cgal"
correspondence_file = "elephant_new_2correspondance-poly-.cgal"

edges_array = read_raw_skeleton(edges_file)
correspondence_array = read_raw_skeleton(correspondence_file)

print(edges_array.shape[0])
print(correspondence_array.shape[0])

#testing the null case
#edges_array = np.array([]).astype(float)
#correspondence_array = np.array([]).astype(float)

edges_array

if len(edges_array) <= 0:
    raise Exception('No skeleton generated for ')

if len(correspondence_array) <= 0:
    raise Exception('No skeleton generated for ')

correspondence_array
